<a href="https://colab.research.google.com/github/aravind2060/Dijkstra_and_kruskal/blob/master/Shortest_path_Algorithm_Dijkstra_and_Kruskal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import heapq
import time

In [16]:
class DijkstraAlgo:
    def __init__(self, vertices, directed=False):
        self.v = vertices
        self.graph = {i: [] for i in range(vertices)}
        self.directed = directed
        self.vertex_map = {}  # Map to store vertex labels to integer mapping
        self.reverse_map = {} # Reverse mapping from integers to labels

    def add_vertex(self, vertex):
        if vertex not in self.vertex_map:
            index = len(self.vertex_map)
            self.vertex_map[vertex] = index
            self.reverse_map[index] = vertex

    def add_edge(self, u, v, w):
        if u not in self.vertex_map:
            self.add_vertex(u)
        if v not in self.vertex_map:
            self.add_vertex(v)
        u_index = self.vertex_map[u]
        v_index = self.vertex_map[v]
        self.graph[u_index].append((v_index, w))
        if not self.directed:
            self.graph[v_index].append((u_index, w))

    def print_graph(self):
        for vertex in self.graph:
            print(f"{self.reverse_map[vertex]} ->", end=" ")
            for edge in self.graph[vertex]:
                print(f"({self.reverse_map[edge[0]]}, {edge[1]})", end=" ")
            print()

    def dijkstra_logic(self, src):
        distances = {vertex: float('infinity') for vertex in self.graph}
        distances[src] = 0
        priority_queue = [(0, src)]
        predecessors = {vertex: None for vertex in self.graph}

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)
            if current_distance > distances[current_vertex]:
                continue

            for neighbor, weight in self.graph[current_vertex]:
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    predecessors[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))

        return distances, predecessors

    def get_path(self, src, dest, predecessors):
        path = []
        step = dest
        while step is not None:
            path.append(step)
            step = predecessors[step]
        path.reverse()
        return [self.reverse_map[node] for node in path if node in self.reverse_map]

In [17]:
class KrushalAlgo:
    def __init__(self, vertices):
        self.V = vertices
        self.graph = []

    def add_edge(self, u, v, w):
        self.graph.append((w, u, v))  # Using tuple for easier sorting by weight

    # Iterative find with path compression
    def find(self, parent, i):
        if parent[i] != i:
            parent[i] = self.find(parent, parent[i])
        return parent[i]

    # Union by rank
    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        if xroot != yroot:
            if rank[xroot] < rank[yroot]:
                parent[xroot] = yroot
            elif rank[xroot] > rank[yroot]:
                parent[yroot] = xroot
            else:
                parent[yroot] = xroot
                rank[xroot] += 1
            return True
        return False

    def kruskal_logic(self):
        result = []
        self.graph.sort()
        parent = list(range(self.V))
        rank = [0] * self.V
        e = 0

        for weight, u, v in self.graph:
            if e == self.V - 1:
                break
            if self.union(parent, rank, u, v):
                result.append((u, v, weight))
                e += 1

        return result

In [18]:
def load_graph_from_file_for_dijkstra(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        first_line = lines[0].split()
        num_vertices = int(first_line[0])
        graph_type = first_line[2]
        directed = graph_type == 'D'

        g = DijkstraAlgo(num_vertices, directed)

        for line in lines[1:-1]:
            u, v, w = line.split()
            g.add_edge(u, v, int(w))

        # Handle optional source node
        source = lines[-1].strip()
        source = g.vertex_map[source] if source in g.vertex_map else None

        return g, source

In [19]:
def load_graph_from_file_for_krushal(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        first_line = lines[0].split()
        num_vertices = int(first_line[0])

        g = KrushalAlgo(num_vertices)
        vertex_map = {}
        vertex_index = 0

        for line in lines[1:]:
            parts = line.strip().split()
            if len(parts) == 3:
                u, v, w = parts[0], parts[1], int(parts[2])

                # Ensure each vertex label is mapped to a unique integer
                if u not in vertex_map:
                    vertex_map[u] = vertex_index
                    vertex_index += 1
                if v not in vertex_map:
                    vertex_map[v] = vertex_index
                    vertex_index += 1

                g.add_edge(vertex_map[u], vertex_map[v], w)
        return g, vertex_map

In [20]:
file_path = '/content/sample_data/input3.txt';
graph, source = load_graph_from_file_for_dijkstra(file_path)
print("Input Graph structure:")
graph.print_graph()
if source is not None:
    print(f"Source vertex: {graph.reverse_map[source]}")
else:
    print("No source vertex specified.")

# Start measuring the time
start_time = time.time()

distances, predecessors = graph.dijkstra_logic(source)

# Calculate the time taken
end_time = time.time()
time_taken = end_time - start_time
print("Time taken for Dijkstra:", time_taken, "seconds\n")

print("Paths and Distances from Source:")
for vertex in graph.graph:
    path = graph.get_path(source, vertex, predecessors)
    print(f"Path from {graph.reverse_map[source]} to {graph.reverse_map[vertex]}: {' -> '.join(path)} with cost {distances[vertex]}")

Input Graph structure:
A -> (B, 1) (C, 2) 
B -> (A, 1) (C, 1) (D, 3) (E, 2) 
C -> (A, 2) (B, 1) (D, 1) (E, 2) 
D -> (B, 3) (C, 1) (E, 4) (F, 3) 
E -> (B, 2) (C, 2) (D, 4) (F, 3) 
F -> (D, 3) (E, 3) 
Source vertex: A
Time taken for Dijkstra: 0.00013780593872070312 seconds

Paths and Distances from Source:
Path from A to A: A with cost 0
Path from A to B: A -> B with cost 1
Path from A to C: A -> C with cost 2
Path from A to D: A -> C -> D with cost 3
Path from A to E: A -> B -> E with cost 3
Path from A to F: A -> C -> D -> F with cost 6


In [21]:

# Specify the path to your input file
file_path = '/content/sample_data/input3.txt';

# Load the graph from the file
graph, vertex_map = load_graph_from_file_for_krushal(file_path)

# Print to verify mapping
print("Vertex Map:", vertex_map)

# Start measuring the time
start_time = time.time()

# Compute the Minimum Spanning Tree using Kruskal's Algorithm
mst = graph.kruskal_logic()

# Calculate the time taken
end_time = time.time()
time_taken = end_time - start_time
print("Time taken for Kruskals Algorithm :", time_taken, "seconds\n")

# Calculate and print the total cost of the Minimum Spanning Tree
total_cost = sum(weight for _, _, weight in mst)
print("Total cost of the Minimum Spanning Tree:", total_cost)

# Print the edges in the Minimum Spanning Tree
print("Edges in the Minimum Spanning Tree:")
for u, v, weight in mst:
    # Mapping integer indexes back to original labels for printing
    u_label = list(vertex_map.keys())[list(vertex_map.values()).index(u)]
    v_label = list(vertex_map.keys())[list(vertex_map.values()).index(v)]
    print(f"({u_label}, {v_label}) - {weight}")


Vertex Map: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5}
Time taken for Krushals Algorithm : 0.00014495849609375 seconds

Total cost of the Minimum Spanning Tree: 8
Edges in the Minimum Spanning Tree:
(A, B) - 1
(B, C) - 1
(C, D) - 1
(B, E) - 2
(D, F) - 3
